**Importing the data from Google Drive**

In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%%capture
# Unzip the data files
!unzip /content/drive/My\ Drive/chest-xray-pneumonia.zip

**Defining the data sources and creating the generators**

In [0]:
# Define the data sources
train_dir = '/content/chest_xray/train'
val_dir = '/content/chest_xray/val'
test_dir = '/content/chest_xray/test'

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# On remplace les valeurs par des valeurs entre 0 et 1
train_datagen = ImageDataGenerator( )
test_datagen = ImageDataGenerator( )

# Create batches
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    batch_size = 12,
                                                    class_mode = 'binary',
                                                    shuffle = True,
                                                    target_size = (2090,1858))

val_generator = train_datagen.flow_from_directory(val_dir, 
                                                    batch_size = 12,
                                                    class_mode = 'binary',
                                                    target_size = (2090,1858))

test_generator = test_datagen.flow_from_directory(test_dir, 
                                                    batch_size = 12,
                                                    class_mode = 'binary',
                                                    target_size = (2090,1858))

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


**Building the CNN model in keras**

In [5]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', input_shape = (2090,1858,3), strides=(4, 4)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 522, 464, 16)      784       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 261, 232, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 129, 115, 16)      4112      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 57, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 31, 27, 16)        4112      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 13, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 5, 32)          8

In [0]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer = RMSprop (lr = 0.001),
             loss = 'binary_crossentropy',
             metrics = ['acc'])

**Training the model**

In [7]:
history = model.fit(train_generator, 
                              validation_data = val_generator,
                              steps_per_epoch = len(train_generator),
                              epochs = 5,
                              validation_steps = len(val_generator),
                              verbose = 1)

Epoch 1/5
435/435 [==============================] - 312s 717ms/step - loss: 0.4782 - acc: 0.8499 - val_loss: 1.0732 - val_acc: 0.6875
Epoch 2/5
435/435 [==============================] - 309s 711ms/step - loss: 0.2325 - acc: 0.9149 - val_loss: 0.5995 - val_acc: 0.8125
Epoch 3/5
435/435 [==============================] - 310s 713ms/step - loss: 0.1550 - acc: 0.9421 - val_loss: 0.2395 - val_acc: 0.9375
Epoch 4/5
435/435 [==============================] - 294s 676ms/step - loss: 0.1423 - acc: 0.9528 - val_loss: 0.4849 - val_acc: 0.8750
Epoch 5/5
435/435 [==============================] - 292s 671ms/step - loss: 0.1160 - acc: 0.9605 - val_loss: 0.2138 - val_acc: 0.9375


We used a small number of epochs to see what happens and to keep the training time low.
Nevertheless, he model seems to be super performant.
Let's see what happens when we use it to predict the test dataset.



**Evaluating the performances**

In [8]:
# evaluate loaded model on test data
score = model.evaluate(test_generator, steps = len(test_generator), verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

52/52 [==============================] - 33s 631ms/step - loss: 2.3658 - acc: 0.6987
acc: 69.87%


The results are not so good, especially compared to results on the validation data. 
The validation set provided contains only 16 images, which seems to be too small compared to the training and test.
We are going to move some files around to adjust the proportions.

In [0]:
import os
def move_from_to_n(src_dir,dst_dir, n):
    # move files, no duplicates!
    fs = os.listdir(src_dir)
    assert len(fs) >= n
    i=0
    targ = fs[-n:]
    # print('targ', targ)
    for f in targ:
        # print('move_from_to_n', i, f, src_dir, dst_dir)
        os.rename(src_dir+'/'+f, dst_dir+'/'+f)
        i+=1
    print('copied',i,src_dir,dst_dir)

In [0]:
%%capture
# we move files from the training dataset to the validation one
move_from_to_n('/content/chest_xray/train/NORMAL', '/content/chest_xray/val/NORMAL', 300)
move_from_to_n('/content/chest_xray/train/PNEUMONIA', '/content/chest_xray/val/PNEUMONIA', 300)

In [11]:
# On remplace les valeurs par des valeurs entre 0 et 1
train_datagen_new = ImageDataGenerator( )
test_datagen_new = ImageDataGenerator( )

# Create batches
train_generator_new = train_datagen_new.flow_from_directory(train_dir, 
                                                    batch_size = 12,
                                                    class_mode = 'binary',
                                                    shuffle = True,
                                                    target_size = (2090,1858))

val_generator_new = train_datagen_new.flow_from_directory(val_dir, 
                                                    batch_size = 12,
                                                    class_mode = 'binary',
                                                    target_size = (2090,1858))

test_generator_new = test_datagen_new.flow_from_directory(test_dir, 
                                                    batch_size = 12,
                                                    class_mode = 'binary',
                                                    target_size = (2090,1858))

Found 4616 images belonging to 2 classes.
Found 616 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


To see the effect of this modification, let's see the training results with the same model.

In [0]:
model_new_data = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', input_shape = (2090,1858,3), strides=(4, 4)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [0]:
model_new_data.compile(optimizer = RMSprop (lr = 0.001),
             loss = 'binary_crossentropy',
             metrics = ['acc'])

In [0]:
history = model_new_data.fit(train_generator_new, 
                              validation_data = val_generator_new,
                              steps_per_epoch = len(train_generator_new),
                              epochs = 5,
                              validation_steps = len(val_generator_new),
                              verbose = 1)

Epoch 1/5
385/385 [==============================] - 319s 828ms/step - loss: 0.6441 - acc: 0.8345 - val_loss: 1.2641 - val_acc: 0.6429
Epoch 2/5
385/385 [==============================] - 321s 834ms/step - loss: 0.2403 - acc: 0.9123 - val_loss: 0.5757 - val_acc: 0.7955
Epoch 3/5
385/385 [==============================] - 324s 841ms/step - loss: 0.1695 - acc: 0.9391 - val_loss: 0.1957 - val_acc: 0.9318
Epoch 4/5
385/385 [==============================] - 326s 847ms/step - loss: 0.1768 - acc: 0.9372 - val_loss: 0.6793 - val_acc: 0.7987
Epoch 5/5
385/385 [==============================] - 325s 843ms/step - loss: 0.1338 - acc: 0.9573 - val_loss: 0.1137 - val_acc: 0.9578


It seems that there is a lot of variance in the validation results : let's use increase the number of epochs.

In [0]:
model_new_data_ex = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', input_shape = (2090,1858,3), strides=(4, 4)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [0]:
model_new_data_ex.compile(optimizer = RMSprop (lr = 0.001),
             loss = 'binary_crossentropy',
             metrics = ['acc'])

In [0]:
history = model_new_data_ex.fit(train_generator_new, 
                              validation_data = val_generator_new,
                              steps_per_epoch = len(train_generator_new),
                              epochs = 15,
                              validation_steps = len(val_generator_new),
                              verbose = 1)

Epoch 1/15
385/385 [==============================] - 344s 894ms/step - loss: 0.4135 - acc: 0.8605 - val_loss: 0.3304 - val_acc: 0.8945
Epoch 2/15
385/385 [==============================] - 330s 858ms/step - loss: 0.2097 - acc: 0.9055 - val_loss: 0.2893 - val_acc: 0.8685
Epoch 3/15
385/385 [==============================] - 331s 861ms/step - loss: 0.1896 - acc: 0.9214 - val_loss: 0.3489 - val_acc: 0.8604
Epoch 4/15
385/385 [==============================] - 333s 864ms/step - loss: 0.1655 - acc: 0.9302 - val_loss: 0.4039 - val_acc: 0.8604
Epoch 5/15
385/385 [==============================] - 331s 860ms/step - loss: 0.1333 - acc: 0.9497 - val_loss: 0.4431 - val_acc: 0.8847
Epoch 6/15
385/385 [==============================] - 334s 867ms/step - loss: 0.1553 - acc: 0.9454 - val_loss: 0.4577 - val_acc: 0.8133
Epoch 7/15
385/385 [==============================] - 331s 859ms/step - loss: 0.1292 - acc: 0.9578 - val_loss: 0.3949 - val_acc: 0.9123
Epoch 8/15
385/385 [============================

We could push it further but it seems that the results aren't going to get better.
Let's put some more parameters into the training process to make the best out of it.

In [0]:
import tensorflow as tf

model_new_data_pm = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', input_shape = (2090,1858,3), strides=(4, 4)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (4,4), activation = 'relu', strides=(2, 2)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [0]:
from tensorflow.keras.optimizers import RMSprop

model_new_data_pm.compile(optimizer = RMSprop (lr = 0.001),
             loss = 'binary_crossentropy',
             metrics = ['acc'])

In [0]:
# Here we define when to stop the training, the version of the model to keep at the end (checkpoint) 
# and the evolution of the learning rate during the training
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
earlyStopping = EarlyStopping(monitor='val_loss', patience=6, verbose=0, mode='min')
mcp_save = ModelCheckpoint('model.h5', save_best_only=True, monitor='val_loss', mode='min')

In [0]:
# We raise the number of epochs to let the other parameters do their job
history = model_new_data_pm.fit(train_generator_new, 
                              validation_data = val_generator_new,
                              steps_per_epoch = len(train_generator_new),
                              epochs = 50,
                              validation_steps = len(val_generator_new),
                              callbacks=[earlyStopping, mcp_save],
                              verbose = 1)

Epoch 1/50
385/385 [==============================] - 333s 866ms/step - loss: 0.5914 - acc: 0.8438 - val_loss: 0.5643 - val_acc: 0.7630 - lr: 0.0010
Epoch 2/50
385/385 [==============================] - 332s 863ms/step - loss: 0.2594 - acc: 0.9079 - val_loss: 0.2293 - val_acc: 0.9286 - lr: 0.0010
Epoch 3/50
385/385 [==============================] - 342s 889ms/step - loss: 0.1997 - acc: 0.9305 - val_loss: 0.2851 - val_acc: 0.8815 - lr: 0.0010
Epoch 4/50
385/385 [==============================] - 337s 875ms/step - loss: 0.1737 - acc: 0.9402 - val_loss: 0.2720 - val_acc: 0.9383 - lr: 0.0010
Epoch 5/50
385/385 [==============================] - 324s 842ms/step - loss: 0.1543 - acc: 0.9528 - val_loss: 0.3369 - val_acc: 0.9010 - lr: 0.0010
Epoch 6/50
385/385 [==============================] - ETA: 0s - loss: 0.1379 - acc: 0.9558
Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
385/385 [==============================] - 344s 894ms/step - loss: 0.1379 - acc: 0.

**Saving the model and evaluating it**

In [0]:
# serialize model to JSON
model_json = model_new_data_pm.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
#model.save_weights("model.h5")

print("Saved model to disk")

Saved model to disk


**Load a model and its weights from files**

In [0]:
from tensorflow.keras.models import model_from_json
# load json and create model
json_file = open('/content/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/model.h5")
print("Loaded model from disk")

Loaded model from disk


In [0]:
# Compile the model
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

**Evaluating the models**

In [0]:
# evaluate loaded model on test data
score = loaded_model.evaluate(test_generator_new, steps = len(test_generator_new), verbose=1)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

52/52 [==============================] - 36s 693ms/step - loss: 1.1724 - accuracy: 0.7756
accuracy: 77.56%


In [0]:
# evaluate loaded model on test data
score = model_new_data_pm.evaluate(test_generator_new, steps = len(test_generator_new), verbose=1)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

52/52 [==============================] - 36s 700ms/step - loss: 1.9539 - acc: 0.7660
accuracy: 76.60%
